# Chain Of Thought Reasoning

In [ ]:
import os

if os.environ.get("TOGETHER_API_KEY") is None:
   os.environ["TOGETHER_API_KEY"] = "paste_your_api_key_here"

In [ ]:
!pip3 install langchain langchain_openai

In [ ]:
import os
from langchain_openai import ChatOpenAI

model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT"
#model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
#model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-Instruct-v0.2"
#model_name = "openchat/openchat-3.5-1210"
#model_name = "Qwen/Qwen1.5-72B-Chat"

print("Provider: TogetherAI")
print("Model: " + model_name)

llm=ChatOpenAI(model_name=model_name,
               openai_api_key=os.environ.get("TOGETHER_API_KEY"),
               openai_api_base="https://api.together.xyz/v1/")

In [ ]:
import textwrap
from langchain_core.prompts import ChatPromptTemplate

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    text = text.lstrip()
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []
    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines
    return '\n'.join(wrapped_lines)

def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(wrap_text(prompt))
    print("")
    print("Response: ")
    print(response)

def print_messages_and_response(messages, response):
    prompt = ChatPromptTemplate(messages=messages)
    print_prompt_and_response(prompt.format(), response)

def get_completion(prompt, temperature=0.0):
    response = llm.invoke(prompt, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

def get_completion_messages(messages, temperature=0.0):
    response=llm.invoke(messages, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

## Chain of Thought Reasoning - Customer Queries

In [ ]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with the tags <query> and </query>

Step 1:{delimiter} First decide whether the user is
asking a question about a specific product or products.
Product cateogry doesn't count.

Step 2:{delimiter} If the user is asking about
specific products, identify whether
the products are in the following list.
All available products:
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products
in the list above, list any assumptions that the
user is making in their
message e.g. that Laptop X is bigger than
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions,
figure out whether the assumption is true based on your
product information.

Step 5:{delimiter}: First, politely correct the
customer's incorrect assumptions if applicable.
Only mention or reference products in the list of
5 available products, as these are the only 5
products that the store sells.
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [ ]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive
than the TechPro Desktop"""

from langchain_core.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(system_message),
    HumanMessage("<query>"+user_message+"</query>")
]

response = get_completion_messages(messages)
print_messages_and_response(messages, response)